In [2]:
import pandas as pd
import numpy as np
from pandas_summary import DataFrameSummary
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [3]:
# pip install pyarrow

In [4]:
df_train = pd.read_feather('all_preprocessed_train.fth')
df_test = pd.read_feather('all_preprocessed_test.fth')

#solo vamos a normalizar las continuas para dps hacer entity embedding
df_train_norm = pd.read_feather('all_preprocessed_train.fth')
df_test_norm = pd.read_feather('all_preprocessed_test.fth')

In [5]:
df_train.shape, df_test.shape

((1017209, 60), (41088, 59))

In [6]:
display(df_train.head().T)

,0,1,2,3,4
Store,1,2,3,4,5
DayOfWeek,5,5,5,5,5
Date,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00
Sales,5263,6064,8314,13995,4822
Customers,555,625,821,1498,559
Open,1,1,1,1,1
Promo,1,1,1,1,1
StateHoliday,0,0,0,0,0
SchoolHoliday,1,1,1,1,1
Year,2015,2015,2015,2015,2015


In [7]:
# Verificación de que se hicieron bien las cosas en esta también
set(df_train.columns) - set(df_test.columns)

{'Customers', 'Sales'}

In [8]:
(df_train['CompetitionDistance'].isna()).sum()

0

In [9]:
df_train.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week', 'Day',
       'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'Promo2Since',
       'State', 'trend', 'trend_DE', 'Max_TemperatureC', 'Mean_TemperatureC',
       'Min_TemperatureC', 'Mean_VisibilityKm', 'Precipitationmm',
       'CloudCover', 'Events', 'StateName', 'CompetitionDaysOpen',
       'CompetitionMonthsOpen', 'Promo2Days', 'Promo2Weeks',
       'StateHoliday_bool', 'Promo_y', 'StateHoliday_bool_y',
       'SchoolHoliday_y', 'AfterSchoolHoliday', 'BeforeSchoolHoliday',
       'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'AfterPromo',
       'BeforePromo', 'AfterSchoolHoliday_y', 'BeforeSchoolHoliday_y',
       'AfterStateHoliday_bool_y', 'BeforeStateHoliday_bool_y', 'AfterPromo_y',
       'Befor

cat
'Store', 'DayOfWeek', 'Date','Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week', 'Day',
       'StoreType', 'Assortment','CompetitionDaysOpen',
       'CompetitionMonthsOpen','Promo2Days' ,'CloudCover', 'Events', 'StateName',

cont
'CompetitionDistance','trend', 'trend_DE', 'Max_TemperatureC', 'Mean_TemperatureC',
       'Min_TemperatureC', 'Mean_VisibilityKm', 'Precipitationmm','Promo2Days', 'StateHoliday_bool_bw', 'Promo_bw', 'SchoolHoliday_bw',
       'StateHoliday_bool_fw', 'Promo_fw', 'SchoolHoliday_fw'


## Categorical vars

In [14]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 'Promo', 'SchoolHoliday', 'Open',
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw']

In [15]:
DataFrameSummary(df_train[cat_vars]).summary()

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,Promo,SchoolHoliday,...,Promo2SinceYear,State,Week,Events,Promo_fw,Promo_bw,SchoolHoliday_fw,SchoolHoliday_bw,StateHoliday_bool_fw,StateHoliday_bool_bw
count,1017209.0,1017209.0,1017209.0,1017209.0,1017209.0,NaN,1017209.0,1017209.0,1017209.0,1017209.0,...,1017209.0,NaN,1017209.0,NaN,1017209.0,1017209.0,1017209.0,1017209.0,1017209.0,1017209.0
mean,558.429727,3.998341,2013.832292,5.846762,15.70279,NaN,20.207503,22.860599,0.381515,0.178647,...,2000.888651,NaN,23.615515,NaN,2.670601,2.648679,1.230985,1.232767,0.206798,0.213673
std,321.908651,1.997391,0.777396,3.326097,8.787638,NaN,7.959786,6.723693,0.485759,0.383056,...,10.93983,NaN,14.433381,NaN,1.902708,1.910292,2.055302,2.053886,0.470661,0.475514
min,1.0,1.0,2013.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0,...,1990.0,NaN,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
25%,280.0,2.0,2013.0,3.0,8.0,NaN,24.0,25.0,0.0,0.0,...,1990.0,NaN,11.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0
50%,558.0,4.0,2014.0,6.0,16.0,NaN,24.0,25.0,0.0,0.0,...,2009.0,NaN,22.0,NaN,3.0,3.0,0.0,0.0,0.0,0.0
75%,838.0,6.0,2014.0,8.0,23.0,NaN,24.0,25.0,1.0,0.0,...,2012.0,NaN,35.0,NaN,5.0,5.0,2.0,2.0,0.0,0.0
max,1115.0,7.0,2015.0,12.0,31.0,NaN,24.0,25.0,1.0,1.0,...,2015.0,NaN,52.0,NaN,5.0,5.0,7.0,7.0,2.0,2.0
counts,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,...,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209
uniques,1115,7,3,12,31,4,25,26,2,2,...,8,12,52,22,6,6,8,8,3,3


In [16]:
DataFrameSummary(df_train[cat_vars]).summary().loc[['uniques', 'types', 'missing']].T

,uniques,types,missing
Store,1115,numeric,0
DayOfWeek,7,numeric,0
Year,3,numeric,0
Month,12,numeric,0
Day,31,numeric,0
StateHoliday,4,categorical,0
CompetitionMonthsOpen,25,numeric,0
Promo2Weeks,26,numeric,0
Promo,2,bool,0
SchoolHoliday,2,bool,0


In [ ]:
# Reemplaza nulls por "" en categoricas
# for v in cat_vars: df_train.loc[df_train[v].isnull(), v] = ""

In [ ]:
# for v in cat_vars: df_test.loc[df_test[v].isnull(), v] = ""

# Continuos Vars

In [21]:
contin_vars = ['CompetitionDistance', 'CompetitionDaysOpen','CompetitionMonthsOpen',
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC','CloudCover', 'trend', 'trend_DE',
   ]

In [22]:
df_train.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week', 'Day',
       'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'Promo2Since',
       'State', 'trend', 'trend_DE', 'Max_TemperatureC', 'Mean_TemperatureC',
       'Min_TemperatureC', 'Mean_VisibilityKm', 'Precipitationmm',
       'CloudCover', 'Events', 'StateName', 'CompetitionDaysOpen',
       'CompetitionMonthsOpen', 'Promo2Days', 'Promo2Weeks',
       'StateHoliday_bool', 'Promo_y', 'StateHoliday_bool_y',
       'SchoolHoliday_y', 'AfterSchoolHoliday', 'BeforeSchoolHoliday',
       'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'AfterPromo',
       'BeforePromo', 'AfterSchoolHoliday_y', 'BeforeSchoolHoliday_y',
       'AfterStateHoliday_bool_y', 'BeforeStateHoliday_bool_y', 'AfterPromo_y',
       'Befor

In [23]:
DataFrameSummary(df_train[contin_vars]).summary().loc[['uniques', 'types', 'missing']].T

,uniques,types,missing
CompetitionDistance,654,numeric,0
CompetitionDaysOpen,8748,numeric,0
CompetitionMonthsOpen,25,numeric,0
Max_TemperatureC,50,numeric,0
Mean_TemperatureC,45,numeric,0
Min_TemperatureC,40,numeric,0
CloudCover,9,numeric,0
trend,67,numeric,0
trend_DE,38,numeric,0


In [24]:
# Reemplaza nulls por 0 en continuas
# for v in contin_vars: df_train.loc[df_train[v].isnull(), v] = 0
# for v in contin_vars: df_test.loc[df_test[v].isnull(), v] = 0

# Normalización en continuas y Encode en Categoricas

In [25]:
# pip install sklearn_pandas
from sklearn_pandas import DataFrameMapper

In [26]:
cat_maps = [(o, LabelEncoder()) for o in cat_vars]
contin_maps = [([o], StandardScaler()) for o in contin_vars]

In [27]:
mapper_cat = DataFrameMapper(cat_maps)
_ = mapper_cat.fit(df_train)

In [28]:
# Assortment es la novena posición en cat_maps
N = 10
print(list(zip(df_train['Assortment'].values[:N], mapper_cat.transform(df_train)[:,9][:N])))
print(list(zip(df_train['Events'].values[:N], mapper_cat.transform(df_train)[:,13][:N])))
print(list(zip(df_train['Year'].values[:N], mapper_cat.transform(df_train)[:,6][:N])))

[('a', 1), ('a', 1), ('a', 1), ('c', 1), ('a', 1), ('a', 1), ('c', 1), ('a', 1), ('c', 1), ('a', 1)]
[('Fog', 0), ('Fog', 2), ('Fog', 2), ('Sunny', 0), ('Sunny', 0), ('Sunny', 0), ('Rain', 0), ('Rain', 0), ('Fog', 0), ('Sunny', 0)]
[(2015, 24), (2015, 24), (2015, 24), (2015, 24), (2015, 3), (2015, 19), (2015, 24), (2015, 9), (2015, 24), (2015, 24)]


In [29]:
mapper_cont = DataFrameMapper(contin_maps)
_ = mapper_cont.fit(df_train)

In [30]:
N = 10
print(df_train['CompetitionDistance'].values[:N])
print(mapper_cont.transform(df_train)[:, 0][:N])

[ 1270.   570. 14130.   620. 29910.   310. 24000.  7520.  2030.  3160.]
[-0.51104292 -0.59341201  1.00219483 -0.5875285   2.85902934 -0.62400624
  2.16359893  0.22439533 -0.42161363 -0.28864639]


In [31]:
# La hacemos con todas ahora
df_train[cat_vars] = mapper_cat.transform(df_train)
df_test[cat_vars] = mapper_cat.transform(df_test)

In [32]:
df_train[contin_vars] = mapper_cont.transform(df_train)
df_test[contin_vars] = mapper_cont.transform(df_test)

In [33]:
df_train_norm[contin_vars] = mapper_cont.transform(df_train_norm)
df_test_norm[contin_vars] = mapper_cont.transform(df_test_norm)

In [50]:
df_train_norm.head().T

,0,1,2,3,4
Store,1,2,3,4,5
DayOfWeek,5,5,5,5,5
Date,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00
Sales,5263,6064,8314,13995,4822
Customers,555,625,821,1498,559
Open,1,1,1,1,1
Promo,1,1,1,1,1
StateHoliday,0,0,0,0,0
SchoolHoliday,1,1,1,1,1
Year,2015,2015,2015,2015,2015


In [35]:
DataFrameSummary(df_train[cat_vars]).summary().loc[['uniques', 'types']]

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,Promo,SchoolHoliday,...,Promo2SinceYear,State,Week,Events,Promo_fw,Promo_bw,SchoolHoliday_fw,SchoolHoliday_bw,StateHoliday_bool_fw,StateHoliday_bool_bw
uniques,1115,7,3,12,31,4,25,26,2,2,...,8,12,52,22,6,6,8,8,3,3
types,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,bool,bool,...,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric


# Final processing

In [36]:
df_train.shape

(1017209, 60)

In [37]:
df_sales = df_train[df_train.Sales!=0]

In [38]:
df_sales_norm = df_train[df_train_norm.Sales!=0]

In [39]:
df_sales.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week', 'Day',
       'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'Promo2Since',
       'State', 'trend', 'trend_DE', 'Max_TemperatureC', 'Mean_TemperatureC',
       'Min_TemperatureC', 'Mean_VisibilityKm', 'Precipitationmm',
       'CloudCover', 'Events', 'StateName', 'CompetitionDaysOpen',
       'CompetitionMonthsOpen', 'Promo2Days', 'Promo2Weeks',
       'StateHoliday_bool', 'Promo_y', 'StateHoliday_bool_y',
       'SchoolHoliday_y', 'AfterSchoolHoliday', 'BeforeSchoolHoliday',
       'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'AfterPromo',
       'BeforePromo', 'AfterSchoolHoliday_y', 'BeforeSchoolHoliday_y',
       'AfterStateHoliday_bool_y', 'BeforeStateHoliday_bool_y', 'AfterPromo_y',
       'Befor

In [40]:
df_test.reset_index(inplace=True)
df_test_norm.reset_index(inplace=True)

In [41]:
df_sales.reset_index(inplace=True)
df_sales_norm.reset_index(inplace=True)

In [42]:
df_sales.to_feather('train_normalized_data.fth')

In [43]:
df_test.to_feather('test_normalized_data.fth')

In [44]:
df_sales_norm.to_feather('train_only_normalized_data.fth')

In [45]:
df_test_norm.to_feather('test_only_normalized_data.fth')

In [46]:
(df_test['Open'] == 0).sum()

5984

In [47]:
(df_sales['Open'] == 0).sum()

0

In [48]:
(df_train['CompetitionDistance'].isna()).sum()

0

In [49]:
display(joined_train.head().T)

NameError: name 'joined_train' is not defined

In [ ]:
good_cropped_train=joined_train.drop(columns=['StateHoliday', 'BeforeStateHoliday_bool_y','AfterPromo_y', 'BeforePromo_y', 'Day_y', 'file_DE', 'week_DE', 'Date_DE', 'State_DE', 'Month_DE', 'Day_DE', 'file_y', 'Year_y', 'Week_y', 'CompetitionOpenSince'])
good_cropped_train=joined_test.drop(columns=['file', 'week', 'Date_y', 'Month_y', 'Day_y', 'file_DE', 'week_DE', 'Date_DE', 'State_DE', 'Month_DE', 'Day_DE', 'file_y', 'Year_y', 'Week_y', 'CompetitionOpenSince' ])